# ディープラーニングフレームワーク2

【問題1】公式チュートリアルモデルを分担して実行

### XLAによるCIFAR-10の分類

XLAとは：<br>
線形代数の演算に特化したコンパイラで、XLAを使うことでTensorFlowの演算を最適化し、メモリ使用量、性能、サーバやモバイル環境での移植性の面での改善が期待できます。現在のところ、ほとんどのユーザにとってXLAを使うことによる大きな恩恵は得られないかもしれませんが、just-in-time (JIT) コンパイル機能 や ahead-of-time (AOT) コンパイル機能 を通して、実験的にXLAを使っていただくのは歓迎です。ただし、新たなハードウエアアクセラレータの開発者については、XLAを試してみることをおすすめします。
引用サイト：https://www.tensorflow.org/xla/architecture?hl=ja<br>
<br>
CIFAR-10とは：<br>
ラベル「0」： airplane（飛行機）<br>
ラベル「1」： automobile（自動車）<br>
ラベル「2」： bird（鳥）<br>
ラベル「3」： cat（猫）<br>
ラベル「4」： deer（鹿）<br>
ラベル「5」： dog（犬）<br>
ラベル「6」： frog（カエル）<br>
ラベル「7」： horse（馬）<br>
ラベル「8」： ship（船）<br>
ラベル「9」： truck（トラック）<br>
という10種類の「物体カラー写真」（乗り物や動物など）の画像データセットである（図1、Alex Krizhevsky氏／Vinod Nair氏／Geoffrey Hinton氏によって、オブジェクト認識用画像データセット「80 Million Tiny Images」から収集されて作成されたサブセットである）。<br>
引用サイト：https://www.atmarkit.co.jp/ait/articles/2006/10/news021.html<br>

In [5]:
import tensorflow as tf

# Check that GPU is available: cf. https://colab.research.google.com/notebooks/gpu.ipynb
# assert(tf.test.gpu_device_name())

tf.keras.backend.clear_session()
tf.config.optimizer.set_jit(False) # Start with XLA disabled.

def load_data():
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
    x_train = x_train.astype('float32') / 256
    x_test = x_test.astype('float32') / 256

    # Convert class vectors to binary class matrices.
    y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
    y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)
    return ((x_train, y_train), (x_test, y_test))

(x_train, y_train), (x_test, y_test) = load_data()

170500096/170498071 [==============================] - 45s 0us/step


In [15]:
x_train.shape

(50000, 32, 32, 3)

In [6]:
def generate_model():
    return tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Conv2D(32, (3, 3)),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(64, (3, 3), padding='same'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Conv2D(64, (3, 3)),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10),
    tf.keras.layers.Activation('softmax')
  ])

model = generate_model()

In [7]:
def compile_model(model):
    opt = tf.keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)
    model.compile(loss='categorical_crossentropy',
                optimizer=opt,
                metrics=['accuracy'])
    return model

model = compile_model(model)

def train_model(model, x_train, y_train, x_test, y_test, epochs=25):
    model.fit(x_train, y_train, batch_size=256, epochs=epochs, validation_data=(x_test, y_test), shuffle=True)

def warmup(model, x_train, y_train, x_test, y_test):
    # Warm up the JIT, we do not wish to measure the compilation time.
    initial_weights = model.get_weights()
    train_model(model, x_train, y_train, x_test, y_test, epochs=1)
    model.set_weights(initial_weights)

warmup(model, x_train, y_train, x_test, y_test)
%time train_model(model, x_train, y_train, x_test, y_test)

scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

196/196 [==============================] - 265s 1s/step - loss: 1.9921 - accuracy: 0.2635 - val_loss: 1.7319 - val_accuracy: 0.3839
Epoch 1/25
196/196 [==============================] - 224s 1s/step - loss: 2.0864 - accuracy: 0.2244 - val_loss: 1.8486 - val_accuracy: 0.3425
Epoch 2/25
196/196 [==============================] - 201s 1s/step - loss: 1.7604 - accuracy: 0.3605 - val_loss: 1.6426 - val_accuracy: 0.4091
Epoch 3/25
196/196 [==============================] - 175s 894ms/step - loss: 1.6556 - accuracy: 0.3992 - val_loss: 1.5569 - val_accuracy: 0.4360
Epoch 4/25
196/196 [==============================] - 178s 907ms/step - loss: 1.5912 - accuracy: 0.4231 - val_loss: 1.4977 - val_accuracy: 0.4543
Epoch 5/25
196/196 [==============================] - 176s 896ms/step - loss: 1.5382 - accuracy: 0.4442 - val_loss: 1.4594 - val_accuracy: 0.4732
Epoch 6/25
196/196 [==============================] - 176s 898ms/step - loss: 1.4904 - accuracy: 0.4611 - val_loss: 1.3860 - val_accuracy: 0.498

In [8]:
# We need to clear the session to enable JIT in the middle of the program.
tf.keras.backend.clear_session()
tf.config.optimizer.set_jit(True) # Enable XLA.
model = compile_model(generate_model())
(x_train, y_train), (x_test, y_test) = load_data()

warmup(model, x_train, y_train, x_test, y_test)
%time train_model(model, x_train, y_train, x_test, y_test)

196/196 [==============================] - 146s 745ms/step - loss: 2.0688 - accuracy: 0.2335 - val_loss: 1.9059 - val_accuracy: 0.3072
Epoch 1/25
196/196 [==============================] - 142s 727ms/step - loss: 2.1339 - accuracy: 0.2069 - val_loss: 1.9553 - val_accuracy: 0.3047
Epoch 2/25
196/196 [==============================] - 141s 722ms/step - loss: 1.8309 - accuracy: 0.3403 - val_loss: 1.7289 - val_accuracy: 0.3817
Epoch 3/25
196/196 [==============================] - 142s 722ms/step - loss: 1.6955 - accuracy: 0.3896 - val_loss: 1.7067 - val_accuracy: 0.3886
Epoch 4/25
196/196 [==============================] - 145s 741ms/step - loss: 1.6116 - accuracy: 0.4163 - val_loss: 1.5962 - val_accuracy: 0.4266
Epoch 5/25
196/196 [==============================] - 141s 721ms/step - loss: 1.5466 - accuracy: 0.4400 - val_loss: 1.4503 - val_accuracy: 0.4804
Epoch 6/25
196/196 [==============================] - 139s 711ms/step - loss: 1.4900 - accuracy: 0.4629 - val_loss: 1.4172 - val_accura

【問題2】（アドバンス課題）様々な手法を実行

後でalbertを読んでみる<br>
https://github.com/google-research/ALBERT<br>

【問題3】Iris（2値分類）をKerasで学習

In [80]:
tf.keras.backend.clear_session()

In [81]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import tensorflow as tf

iris = load_iris()
data = iris.data
label = iris.target
target_name = iris.target_names
all_data = np.concatenate([data, label.reshape(-1, 1)], axis=1)
print(target_name)

train_df = pd.DataFrame(all_data)
train = train_df.iloc[:, :]
train.rename(columns={0:'SepalLengthCm', 1:'SepalWidthCm', 2:'PetalLengthCm', 3:'PetalWidthCm', 4:'Species'}, inplace=True)
train['Species'].replace(0.0, 'Iris-setosa', inplace=True)
train['Species'].replace(1.0, 'Iris-versicolor', inplace=True)
train['Species'].replace(2.0, 'Iris-virginica', inplace=True)
X = train.loc[50:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = train.loc[50:, ['Species']]

X = np.array(X)
y = np.array(y)
# ラベルを数値に変換
y[y == "Iris-versicolor"] = 0
y[y == "Iris-virginica"] = 1

y = y.astype(np.int64)[:, np.newaxis]
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train2, X_val, y_train2, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

['setosa' 'versicolor' 'virginica']


In [ ]:
X_train.shape = (80, 4)

(None, 80, 4)

In [92]:
def generate_model():
    return tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, input_dim=X_train.shape[1]),
#     tf.keras.layers.Activation('relu'),
#     tf.keras.layers.Dense(128),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(64),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(32),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Activation('sigmoid')
  ])

model = generate_model()


def compile_model(model):
    opt = tf.keras.optimizers.Adam(lr=0.01)
    model.compile(loss='binary_crossentropy',
                optimizer=opt,
                metrics=['accuracy'])
    return model

model = compile_model(model)

def train_model(model, X_train, y_train, X_val, y_val, epochs=100):
    model.fit(X_train, y_train, batch_size=5, epochs=epochs, validation_data=(X_val, y_val), shuffle=True)

def warmup(model, X_train, y_train, X_test, y_test):
    # Warm up the JIT, we do not wish to measure the compilation time.
    initial_weights = model.get_weights()
    train_model(model, X_train, y_train, X_test, y_test, epochs=1)
    model.set_weights(initial_weights)

warmup(model, X_train, y_train, X_test, y_test)
%time train_model(model, X_train, y_train, X_val, y_val)

scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

16/16 [==============================] - 0s 20ms/step - loss: 0.8092 - accuracy: 0.5375 - val_loss: 0.6664 - val_accuracy: 0.5000
Epoch 1/100
16/16 [==============================] - 0s 19ms/step - loss: 0.7224 - accuracy: 0.4750 - val_loss: 0.6267 - val_accuracy: 0.6250
Epoch 2/100
16/16 [==============================] - 0s 9ms/step - loss: 0.6959 - accuracy: 0.5375 - val_loss: 0.6685 - val_accuracy: 0.6250
Epoch 3/100
16/16 [==============================] - 0s 10ms/step - loss: 0.6418 - accuracy: 0.7250 - val_loss: 0.5219 - val_accuracy: 1.0000
Epoch 4/100
16/16 [==============================] - 0s 9ms/step - loss: 0.6140 - accuracy: 0.6500 - val_loss: 0.7652 - val_accuracy: 0.3750
Epoch 5/100
16/16 [==============================] - 0s 9ms/step - loss: 0.5344 - accuracy: 0.6875 - val_loss: 0.2949 - val_accuracy: 1.0000
Epoch 6/100
16/16 [==============================] - 0s 10ms/step - loss: 0.4352 - accuracy: 0.8500 - val_loss: 0.0887 - val_accuracy: 1.0000
Epoch 7/100
16/16 [==

Epoch 55/100
16/16 [==============================] - 0s 17ms/step - loss: 0.0770 - accuracy: 0.9750 - val_loss: 0.0082 - val_accuracy: 1.0000
Epoch 56/100
16/16 [==============================] - 0s 12ms/step - loss: 0.0778 - accuracy: 0.9625 - val_loss: 0.1965 - val_accuracy: 0.8125
Epoch 57/100
16/16 [==============================] - 0s 14ms/step - loss: 0.3678 - accuracy: 0.9000 - val_loss: 0.1837 - val_accuracy: 0.8750
Epoch 58/100
16/16 [==============================] - 0s 9ms/step - loss: 0.2452 - accuracy: 0.9250 - val_loss: 0.0874 - val_accuracy: 0.9375
Epoch 59/100
16/16 [==============================] - 0s 13ms/step - loss: 0.0933 - accuracy: 0.9625 - val_loss: 0.1572 - val_accuracy: 0.9375
Epoch 60/100
16/16 [==============================] - 0s 19ms/step - loss: 0.0676 - accuracy: 0.9750 - val_loss: 0.0233 - val_accuracy: 1.0000
Epoch 61/100
16/16 [==============================] - 0s 10ms/step - loss: 0.1792 - accuracy: 0.9250 - val_loss: 0.1629 - val_accuracy: 0.8750


【問題4】Iris（多値分類）をKerasで学習

In [100]:
# tf.keras.backend.clear_session()
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import tensorflow as tf

iris = load_iris()
data = iris.data
label = iris.target
target_name = iris.target_names
all_data = np.concatenate([data, label.reshape(-1, 1)], axis=1)
print(target_name)

train_df = pd.DataFrame(all_data)
train = train_df
train.rename(columns={0:'SepalLengthCm', 1:'SepalWidthCm', 2:'PetalLengthCm', 3:'PetalWidthCm', 4:'Species'}, inplace=True)
train['Species'].replace(0.0, 'Iris-setosa', inplace=True)
train['Species'].replace(1.0, 'Iris-versicolor', inplace=True)
train['Species'].replace(2.0, 'Iris-virginica', inplace=True)
X = train.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = train.loc[:, ['Species']]

X = np.array(X)
y = np.array(y)
# ラベルを数値に変換
y[y == "Iris-setosa"] = 0
y[y == "Iris-versicolor"] = 1
y[y == "Iris-virginica"] = 2
y = tf.keras.utils.to_categorical(y, num_classes=3)

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train2, X_val, y_train2, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

['setosa' 'versicolor' 'virginica']


In [101]:
y_train

array([[0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1

In [102]:
tf.keras.backend.clear_session()

In [103]:
def generate_model():
    return tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, input_shape=X_train.shape[1:]),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(128),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(64),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(32),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(3),
    tf.keras.layers.Activation('softmax')
  ])

model = generate_model()


def compile_model(model):
    opt = tf.keras.optimizers.RMSprop(lr=0.01)
    model.compile(loss='categorical_crossentropy',
                optimizer=opt,
                metrics=['accuracy'])
    return model

model = compile_model(model)

def train_model(model, X_train2, y_train2, X_val, y_val, epochs=100):
    model.fit(X_train2, y_train2, batch_size=5, epochs=epochs, validation_data=(X_val, y_val), shuffle=True)

def warmup(model, X_train, y_train, X_test, y_test):
    # Warm up the JIT, we do not wish to measure the compilation time.
    initial_weights = model.get_weights()
    train_model(model, X_train, y_train, X_test, y_test, epochs=1)
    model.set_weights(initial_weights)

warmup(model, X_train, y_train, X_test, y_test)
%time train_model(model, X_train2, y_train2, X_val, y_val)

scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

24/24 [==============================] - 0s 16ms/step - loss: 1.0137 - accuracy: 0.5667 - val_loss: 0.4225 - val_accuracy: 0.8000
Epoch 1/100
20/20 [==============================] - 0s 14ms/step - loss: 1.3877 - accuracy: 0.6667 - val_loss: 0.4998 - val_accuracy: 0.6250
Epoch 2/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5092 - accuracy: 0.6458 - val_loss: 0.4202 - val_accuracy: 0.7083
Epoch 3/100
20/20 [==============================] - 0s 5ms/step - loss: 0.7231 - accuracy: 0.7708 - val_loss: 0.5183 - val_accuracy: 0.6250
Epoch 4/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4210 - accuracy: 0.8229 - val_loss: 0.3822 - val_accuracy: 0.9167
Epoch 5/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4223 - accuracy: 0.7812 - val_loss: 0.6223 - val_accuracy: 0.7083
Epoch 6/100
20/20 [==============================] - 0s 5ms/step - loss: 0.3904 - accuracy: 0.8646 - val_loss: 1.6706 - val_accuracy: 0.7083
Epoch 7/100
20/20 [====

20/20 [==============================] - 0s 15ms/step - loss: 0.1909 - accuracy: 0.9792 - val_loss: 0.3570 - val_accuracy: 0.9167
Epoch 58/100
20/20 [==============================] - 0s 9ms/step - loss: 0.1285 - accuracy: 0.9375 - val_loss: 0.5996 - val_accuracy: 0.9167
Epoch 59/100
20/20 [==============================] - 0s 4ms/step - loss: 0.1570 - accuracy: 0.9583 - val_loss: 0.1594 - val_accuracy: 0.9167
Epoch 60/100
20/20 [==============================] - 0s 5ms/step - loss: 0.2681 - accuracy: 0.9479 - val_loss: 0.1886 - val_accuracy: 0.9167
Epoch 61/100
20/20 [==============================] - 0s 7ms/step - loss: 0.1224 - accuracy: 0.9375 - val_loss: 0.1386 - val_accuracy: 0.9167
Epoch 62/100
20/20 [==============================] - 0s 5ms/step - loss: 0.3836 - accuracy: 0.9271 - val_loss: 1.1640 - val_accuracy: 0.7500
Epoch 63/100
20/20 [==============================] - 0s 7ms/step - loss: 0.1177 - accuracy: 0.9688 - val_loss: 0.3738 - val_accuracy: 0.9167
Epoch 64/100
20/20

【問題5】House PricesをKerasで学習

In [52]:
tf.keras.backend.clear_session()
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import tensorflow as tf

DIR_PATH = '/Users/sasanoshouta/Desktop/DIVE_INTO_CODE/python/week3/house-prices-advanced-regression-techniques/'
train_df = pd.read_csv(DIR_PATH+'train.csv')
test_df = pd.read_csv(DIR_PATH+'test.csv')

X = train_df[['GrLivArea', 'YearBuilt']]
y = train_df[['SalePrice']]
X['GrLivArea'] = X['GrLivArea'].apply(np.log)
y['SalePrice'] = y['SalePrice'].apply(np.log)

X = np.array(X)
y = np.array(y)

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train2, X_val, y_train2, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

<ipython-input-52-702553b0d5d4>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['GrLivArea'] = X['GrLivArea'].apply(np.log)
<ipython-input-52-702553b0d5d4>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['SalePrice'] = y['SalePrice'].apply(np.log)


In [53]:
def generate_model():
    return tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, input_shape=X_train.shape[1:]),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(128),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(64),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(32),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(1)
  ])

model = generate_model()


def compile_model(model):
    opt = tf.keras.optimizers.Adam(lr=0.01)
    model.compile(loss='mean_squared_error',
                optimizer=opt)
    return model

model = compile_model(model)

def train_model(model, X_train2, y_train2, X_val, y_val, epochs=100):
    model.fit(X_train2, y_train2, batch_size=5, epochs=epochs, validation_data=(X_val, y_val), shuffle=True)

def warmup(model, X_train, y_train, X_test, y_test):
    # Warm up the JIT, we do not wish to measure the compilation time.
    initial_weights = model.get_weights()
    train_model(model, X_train, y_train, X_test, y_test, epochs=1)
    model.set_weights(initial_weights)

warmup(model, X_train, y_train, X_test, y_test)
%time train_model(model, X_train2, y_train2, X_val, y_val)

scores = model.evaluate(X_test, y_test, verbose=1)
print('Test mse:', scores)
# print('Test accuracy:', scores[1])

234/234 [==============================] - 1s 3ms/step - loss: 1874.5500 - val_loss: 0.1134
Epoch 1/100
187/187 [==============================] - 0s 2ms/step - loss: 1095.1494 - val_loss: 0.1902
Epoch 2/100
187/187 [==============================] - 0s 2ms/step - loss: 0.3445 - val_loss: 0.1262
Epoch 3/100
187/187 [==============================] - 1s 3ms/step - loss: 0.4656 - val_loss: 1.4793
Epoch 4/100
187/187 [==============================] - 0s 2ms/step - loss: 0.5421 - val_loss: 0.5668
Epoch 5/100
187/187 [==============================] - 0s 2ms/step - loss: 0.5961 - val_loss: 0.3431
Epoch 6/100
187/187 [==============================] - 0s 2ms/step - loss: 0.6197 - val_loss: 0.1720
Epoch 7/100
187/187 [==============================] - 1s 3ms/step - loss: 0.6533 - val_loss: 0.1127
Epoch 8/100
187/187 [==============================] - 0s 2ms/step - loss: 0.4072 - val_loss: 0.2100
Epoch 9/100
187/187 [==============================] - 1s 3ms/step - loss: 0.4301 - val_loss: 0.9

【問題6】MNISTをKerasで学習

In [97]:
tf.keras.backend.clear_session()
import numpy as np
import pandas as pd
import tensorflow as tf


def load_mnist():
    (X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
    X_train = X_train.astype('float32') / 256
    X_test = X_test.astype('float32') / 256
    X_train = X_train.reshape(-1, 28, 28, 1)
    X_test = X_test.reshape(-1, 28, 28, 1)

    # Convert class vectors to binary class matrices.
    y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
    y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)
    return ((X_train, y_train), (X_test, y_test))

(X_train, y_train), (X_test, y_test) = load_mnist()

In [99]:
y_train[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

In [ ]:
X_train.shape = (60000, 28, 28)
(sample, h, w, c)

In [78]:
def generate_model():
    return tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:]),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Conv2D(32, (3, 3)),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(64, (3, 3), padding='same'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Conv2D(64, (3, 3)),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10),
    tf.keras.layers.Activation('softmax')
  ])

model = generate_model()

def compile_model(model):
    opt = tf.keras.optimizers.Adam(lr=0.01, decay=1e-6)
    model.compile(loss='categorical_crossentropy',
                optimizer=opt,
                metrics=['accuracy'])
    return model

model = compile_model(model)

def train_model(model, X_train, y_train, X_test, y_test, epochs=25):
    model.fit(X_train, y_train, batch_size=300, epochs=epochs, validation_data=(X_test, y_test), shuffle=True)

def warmup(model, X_train, y_train, X_test, y_test):
    # Warm up the JIT, we do not wish to measure the compilation time.
    initial_weights = model.get_weights()
    train_model(model, X_train, y_train, X_test, y_test, epochs=1)
    model.set_weights(initial_weights)

warmup(model, X_train, y_train, X_test, y_test)
%time train_model(model, X_train, y_train, X_test, y_test)

scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

200/200 [==============================] - 149s 744ms/step - loss: 2.3061 - accuracy: 0.1112 - val_loss: 2.3014 - val_accuracy: 0.1135
Epoch 1/25
200/200 [==============================] - 137s 686ms/step - loss: 0.8151 - accuracy: 0.7266 - val_loss: 0.2419 - val_accuracy: 0.9306
Epoch 2/25
200/200 [==============================] - 145s 727ms/step - loss: 0.3910 - accuracy: 0.8748 - val_loss: 0.1484 - val_accuracy: 0.9566
Epoch 3/25
200/200 [==============================] - 122s 612ms/step - loss: 0.2675 - accuracy: 0.9179 - val_loss: 0.1090 - val_accuracy: 0.9683
Epoch 4/25
200/200 [==============================] - 121s 603ms/step - loss: 0.2099 - accuracy: 0.9350 - val_loss: 0.1252 - val_accuracy: 0.9644
Epoch 5/25
200/200 [==============================] - 120s 601ms/step - loss: 0.1878 - accuracy: 0.9429 - val_loss: 0.0666 - val_accuracy: 0.9825
Epoch 6/25
200/200 [==============================] - 130s 648ms/step - loss: 0.1719 - accuracy: 0.9467 - val_loss: 0.0574 - val_accura

履歴の可視化：https://tensorflow.classcat.com/2018/08/26/tensorflow-tutorials-keras-basic-regression/

【問題7】（アドバンス課題）PyTorchへの書き換え

【問題8】（アドバンス課題）フレームワークの比較